# Parametric Modelling in ISAMBARD

In [ ]:
import itertools
from pprint import pprint

import matplotlib.pyplot as plt
import nglview as nv
import numpy

%matplotlib inline

In [ ]:
import isambard

In [ ]:
def show_ball_and_stick(ampal):
    view = nv.show_text(ampal.pdb)
    view.add_ball_and_stick()
    view.remove_cartoon()
    return view

## What is parametric modelling?

Parametric modelling is a method of creating models of biomolecular structure using a simple geometric description of the shape of the molecule. The molecular structure of the protein backbone enables it to adopt a huge number of conformations, but if we are modelling an amino acid sequence that is likely to be a parametrisable protein fold, we can reduce the complexity of modelling the sequence by using this simplified geometric description.

The most famous example of a parametrisable protein fold is the α-helical coiled coil, which accounts for ~5% of all protein-encoding DNA. This fold consists of two or more α-helices that wrap around each other to form a rope-like super-helical structure. Their structures range from very simple to highly complex, but at the core, all these structures can be described using a simple mathematical model. In the simplest case, only 3 geometric parameters are required to describe the position of all backbone atoms with near atomic precision! These parameters are pitch, radius and interface angle (also known as ϕCα) (Figure 1).

<figure>
    <img src="imgs/figure1.png" alt="Structure of α-helical coiled coil.">
    <figcaption>
        **Figure 1. Structure of α-helical coiled coils.** (A) Helical-wheel diagrams showing
        the projection of residues in the heptad repeat. (B) Helices in a coiled coil pack 
        closely together, forming knobs-into-holes interactions. (C) Coiled coils can be 
        described using three geometric parameters: interface angle (°), radius (Å), and 
        pitch (Å).
    </figcaption>
</figure>

You can get a better feeling for what these parameters represent by playing with [CCBuilder 2.0](http://coiledcoils.chm.bris.ac.uk/ccbuilder2/builder). CCBuilder is a interactive web application for modelling coiled coils, built on top of ISAMBARD. Under the hood it uses a lot of the functionality that we'll be learning about in this tutorial.

## Specifications

Parametric models in ISAMBARD are known as "specifications", a term borrowed from the field of architecture, where a specification is defined as "a detailed description of the design and materials used to make something". There are two types of specifications, one at the `Polymer` level and one at the `Assembly` level. `Polymer` specifications describe regions of regular backbone structure, like an α-helix. `Assembly` specifications describe how regions of secondary structure assemble together.

### The Coiled-Coil Specification

Let's start by building modelling some coiled coils. The very simplest way of creating a model is to build it using typical parameters for a particular oligomeric state of coiled coil.

In [ ]:
dimer = isambard.specifications.CoiledCoil(2)

In [ ]:
show_ball_and_stick(dimer)

In [ ]:
dimer

Parametric models in ISAMBARD are AMPAL objects, just the same as any other structure that you might load in, so you can use the same tools that you encountered in the first tutorial to analyse their structure. They do contain some extra functionality though, such as the attributes and methods required to create the backbone model. Let's take a look at the values of the coiled coil parameters described above.

In [ ]:
print("Radii: {}\nPitch values: {}\nInterface Angles: {}".format(
    dimer.major_radii, dimer.major_pitches, dimer.phi_c_alphas))

Here you can see that there are *n* values for each parameters, where *n* is equal to the oligomeric state. This is because the coiled coil parameters are defined relative to the super-helical axis.

Let's add some sidechains to this structure.

> **Note**<br/>Currently we use [SCWRL4](http://dunbrack.fccc.edu/scwrl4/) to add sidechains to structures, so you need to download a copy if you want to do this on your own computer. It requires a license but is free for academics.

In [ ]:
basis_set_dimer_sequences = [
    'EIAALKQEIAALKKENAALKWEIAALKQ',
    'EIAALKQEIAALKKENAALKWEIAALKQ'
#    gabcdefgabcdefgabcdefgabcdef  <- This is the heptad repeat of this sequence, we'll come back to this
]

In [ ]:
dimer.pack_new_sequences(basis_set_dimer_sequences)

In [ ]:
show_ball_and_stick(dimer)

You can also create any <i>n</i>mer in the same way.

In [ ]:
trimer = isambard.specifications.CoiledCoil(3)
tetramer = isambard.specifications.CoiledCoil(4)
pentamer = isambard.specifications.CoiledCoil(5)
hexamer = isambard.specifications.CoiledCoil(6)

In [ ]:
pprint([dimer, trimer, tetramer, pentamer, hexamer])

Chances are though that you'll want to actually change the parameters used to create the model, it is parametric modelling after all!

### Building with Parameters

While we can directly modify the parameters on the models we created earlier, it's not a particularly convenient method of generating models, so we can use the `from_parameters` class method instead. Let's have a look at the documentation for this.

In [ ]:
help(isambard.specifications.CoiledCoil.from_parameters)

> ### Note
> Don't forget that you can see information on specific functions/classes in a number of ways:
> 1. Check the [API documentation](https://woolfson-group.github.io/isambard/api_reference.html)
> 1. Take a look at the [source code](https://github.com/woolfson-group/isambard/tree/master/isambard)
> 1. Shift+Tab inside the round brackets if you're using Jupyter Notebook
> 1. Use the Python `help` function i.e. `help(isambard.specifications.CoiledCoil.from_parameters)`

Here you can see all the parameters available; oligomeric state is the only required parameter, the others you could leave blank and it will function in pretty much the same as instantiating `CoiledCoil` directly. Let's make a dimer again but explicitly state the parameters.

In [ ]:
dimer_exp_params = isambard.specifications.CoiledCoil.from_parameters(
    2, 28, 5, 225, 283) # i.e. 2 chains, 28 residues, radius 5Å, pitch 225Å, phi-c-alpha 283°

In [ ]:
dimer_exp_params.pack_new_sequences(basis_set_dimer_sequences)

In [ ]:
show_ball_and_stick(dimer_exp_params)

When modelling coiled coils, it can be useful to talk about the interface angle relative to the *a* position residue, and so you adjust its value based on the register of your sequence *i.e.* which heptad position is your first residue (Figure 1). Here's a little dictionary to convert your register to an interface angle at *a*.

In [ ]:
REGISTER_ADJUST = {
    'a': 0,
    'b': 102.8,
    'c': 205.6,
    'd': 308.4,
    'e': 51.4,
    'f': 154.2,
    'g': 257
}

Let's use this to make it easier to think about our sequence. The basis set dimer sequence starts at a *g* position, and usually the *a* position residues sit at around 26° (see Figure 1C).

In [ ]:
dimer_with_reg = isambard.specifications.CoiledCoil.from_parameters(2, 28, 4.8, 160, REGISTER_ADJUST['g']+26)

In [ ]:
dimer_with_reg.pack_new_sequences(basis_set_dimer_sequences)

In [ ]:
show_ball_and_stick(dimer_with_reg)

If we change the register of our sequence we just need to update the adjust value.

In [ ]:
f_dimer_sequences = [
    'QEIAALKKENAALKWEIAALKQEIAALK',
    'QEIAALKKENAALKWEIAALKQEIAALK'
#    fgabcdefgabcdefgabcdefgabcde
]

In [ ]:
f_dimer = isambard.specifications.CoiledCoil.from_parameters(2, 28, 4.8, 160, REGISTER_ADJUST['f']+26)

In [ ]:
f_dimer.pack_new_sequences(f_dimer_sequences)

In [ ]:
show_ball_and_stick(f_dimer)

As you can see, the hydrophobic core is still buried.

> ### Problem
> Try changing the register without updating the sequence. What happens?

## Parametric Power! (and associated responsibility)

The only restriction on the conformations that you can model in ISAMBARD is the underlying geometric description in the specification, which means that you can build any backbone structure that is geometrically possible. This is very powerful as you can model protein structures that have not been observed in nature, and use the models as the basis for design. Let's make a dimer with some weird parameters.

In [ ]:
weird_dimer = isambard.specifications.CoiledCoil.from_parameters(2, 60, 10, 80, 180)

In [ ]:
show_ball_and_stick(weird_dimer)

In [ ]:
weird_trimer = isambard.specifications.CoiledCoil.from_parameters(3, 20, 6, 1000, 278)

In [ ]:
show_ball_and_stick(weird_trimer)

These models have extreme values for parameters and so sequences are highly unlikely to fold into these structures. Rather than restricting the parameters that are available to the user (who am I to judge what you build in your own time!), we provide a range of metrics to evaluate the models you produce.

## Evaluating Models

### Geometric Evaluation

The `analyse_protein` module provides a range of tools to evaluating the backbone geometry of your model. One of the most useful metrics we can use is the residues per turn (RPT) of the  α-helices in the model. At extreme parameter values, RPT can move away out of the range of values observed in α-helices of known protein structures (Figure 2), which indicates that there is backbone strain in the model.

<figure>
    <img src="imgs/figure2.png" alt="Values of residues per turn observed in known structures."
         height="60%" width="60%">
    <figcaption>
        **Figure 2. Values of residues per turn observed in known structures**. Grey bars: Values of residues per 
        turn from helices in known structures. Mean = 3.65 (SD = 0.07). White bars: Values of RPT found in coiled-
        coil crystal structures. Mean = 3.62 (SD = 0.07)
    </figcaption>
</figure>

As you can see, this distribution is very tight, so if your number of residues per turn moves too far away from the mean, it's probably indicating that the model isn't very good. Let measure the RPT values on our models.

In [ ]:
dimer_rpt_values = [isambard.analyse_protein.residues_per_turn(p) for p in dimer]

In [ ]:
pprint(dimer_rpt_values)

`residues_per_turn` takes a `Polypeptide` object and returns a list of RPT values for each residue, so we needed to apply the function to each `Polypeptide` in the dimer `Assembly` (we used a list comprehension to do that, if you're not familiar with the comprehension syntax in Python, have a look at [the docs](https://docs.python.org/3.6/tutorial/datastructures.html#list-comprehensions)). `None` is returned for the last residues as RPT is undefined at the C-terminus.

Let's calculate the average value. First we want to merge the per chain values together and filter out `None`.

In [ ]:
dimer_rpt_values = itertools.chain(*dimer_rpt_values)  # This flattens a list of lists to a single list
dimer_rpt_values = [x for x in dimer_rpt_values if x is not None]  # This removes the Nones

In [ ]:
dimer_rpt_values

In [ ]:
# We can use the numpy package to calculate the means/std
print('Mean = {:.2f}, STD = {:.2f}'.format(numpy.mean(dimer_rpt_values), numpy.std(dimer_rpt_values)))

So the mean is just under 3.6, which is quite low, but well within the distribution of observed RPT values.

> ### Problem
> Calculate the RPT values for the `weird_dimer` and `weird_trimer`. Do they fall inside the observed distributions?

> ### Note
>There are two other useful metrics to evaluate backbone geometry: (a) rise per residue and (b) residues per turn. You can access these directly on the `Polymer` object (try this `dimer[0].rise_per_residue()` and `dimer[0].radii_of_curvature()`). We won't talk about these here but they can also be useful for evaluating structures.

### All-Atom Force Fields

Another useful way to evaluate your structures is with an all-atom scoring function. The BUDE force field from the Sessions Lab in Bristol (see [this paper](http://dx.doi.org/10.1093/comjnl/bxr091) for more details) is currently bundled with ISAMBARD. We like BUDE because it's fast and suits our purposes, but you can use any force field you like. We can calculate the BUDE energies using built-in properties of the AMPAL objects.

In [ ]:
dimer_score = dimer.buff_internal_energy
print(dimer_score)

The dimer we created earlier has a total BUDE energy of -824.24, which is composed of a steric component (St), an energy of desolvation (De) and a charged component (Ch). This returns a BUDE Force Feild (BUFF) score object, which contains lots of information about the score including the individual components, and all the interactions that the make up the score.

> ### Note
> BUDE follows the thermodynamic convention of lower = more favourable.

In [ ]:
dimer_score.total_energy

In [ ]:
dimer_score.steric

In [ ]:
dimer_score.desolvation

In [ ]:
dimer_score.charge

In [ ]:
dimer_score.inter_scores[:5]

The `inter_scores` list contains all the non-zero pairwise atom interaction scores from the structure. Each list item contains a tuple with the pair of atoms and a list of the different BUDE components _i.e._ `[steric, desolvation, charge]`. It can be useful to examine these scores to find clashes or important residues.

In [ ]:
clashes = [x for x in dimer_score.inter_scores if x[1][0] > 0]  # filtering for clashes

In [ ]:
clashes

The third interaction is a clash between 2 oxygen atoms. We can get more information on the residues involved from the atoms themselves.

In [ ]:
(atom1, atom2), score_components = clashes[2]  # we can unpack the interaction to get the atoms

In [ ]:
print(atom1.unique_id, atom1.ampal_parent)
print(atom2.unique_id, atom2.ampal_parent)

We can see here that it is the oxygen atoms on the asparagine side chains in the core are clashing slightly, which you can see in the structure of the first example. This indicates that the parameters used to create this model could be improved for this sequence, so let's do that!

In [ ]:
show_ball_and_stick(dimer)

> ### Problem
> Try and find other interesting interactions, such as the atom pair with the lowest charged interaction.

## Fitting Parameters

When performing parametric modelling, you might not be sure of the parameters that you should use to best model a given sequence, in this case we can fit parameters for a given sequence.

### Grid Scan

The simplest form of parameter fitting is to perform a grid scan where we uniformly sample parameters across a range of allowed values. Let's perform a grid scan of radius vs interface angle for a dimer. First of all we can define the range of parameter values we want to explore.

In [ ]:
radii = numpy.arange(4, 6, 0.2)  # (min, max, step)
interface_angles = numpy.arange(10, 30, 2)

Next we'll write a little function to build a model from radius and interface angle values. We're keeping pitch fixed as this has the smallest overall impact on the model quality.

In [ ]:
def build_dimer(radius, interface_angle):
    sequences = ['EIAALKQEIAALKKENAALKWEIAALKQ', 'EIAALKQEIAALKKENAALKWEIAALKQ']
    gs_dimer = isambard.specifications.CoiledCoil.from_parameters(
        2, 28, radius, 160, REGISTER_ADJUST['g']+interface_angle)
    gs_dimer.pack_new_sequences(sequences)
    return gs_dimer

Finally we can create an array (a 2D matrix) to hold our results using numpy, and we can populate the grid positions with the BUFF score of each of the corresponding models.

In [ ]:
results_array = numpy.empty((len(radii), len(interface_angles)))
for i, radius in enumerate(radii):
    for j, interface_angle in enumerate(interface_angles):
        dimer_model = build_dimer(radius, interface_angle)
        results_array[i][j] = dimer_model.buff_internal_energy.total_energy

We can plot a heatmap of the array to show which parameters have the best scores!

In [ ]:
plt.imshow(results_array)
plt.ylabel('Radius (Å)')
plt.xlabel('Interface Angle (°)')
plt.yticks(range(len(radii)), ['{:.1f}'.format(x) for x in radii])
plt.xticks(range(len(interface_angles)), interface_angles)
plt.colorbar()

The model with a radius of 4.4 Å and interface angle of 18° is the best, let's have a look at the model.

In [ ]:
best_dimer = build_dimer(4.4, 18)

In [ ]:
best_dimer.buff_internal_energy

In [ ]:
show_ball_and_stick(best_dimer)

If you zoom in on the asparagine residues in the core, you should see that they are now forming a nice hydrogen-bonding network. Neat!

> ### Problem
> Try performing a finer grid scan around the best parameters found by the last grid scan. Stick to at most 100 models or it might take a long time to finish.

When the parameter space that you want to explore is small, a grid scan is a good option. You can scale up the model building through parallelisation using the [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) module, based on your available resources. However, as you introduce more parameters, it quickly becomes very inefficient to perform a grid scan, and so other methods of parameter fitting are preferred (see Tutorial 3!).